In [ ]:
!git clone https://github.com/ManchesterBioinference/GPcounts.git

In [ ]:
#You have to modify the requirements.txt file and change the gpflow version to 2.5.2. Otherwise later code fails.
%cd /content/GPcounts
!pip install -r requirements.txt
%cd ..

In [ ]:
!pip install git+https://github.com/markvdw/RobustGP.git

In [ ]:
%cd /content/GPcounts
!python setup.py install
%cd ..

In [1]:
import pandas as pd 
import numpy as np 
import gpflow
import tensorflow as tf
from GPcounts.GPcounts_Module import Fit_GPcounts
from matplotlib import pyplot as plt


2023-03-09 01:45:40.187808: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


###Load data

In [3]:
# spatial expression counts matrix
Y = pd.read_csv('./GPcounts/data/MouseOB/Rep11_MOB_0.csv', index_col=[0])
Y = Y.T[Y.sum(0) >= 3].T # Filter out genes with less than 3 total counts
Y.head()

,Nrf1,Zbtb5,Ccnl1,Lrrfip1,Bbs1,Lix1,Whrn,Ate1,Ubac1,Rab34,...,Mertk,Snord19,1500002O10Rik,Gm5105,H2-Bl,Mir384,Snord91a,Zdhhc23,Fam150a,Hist4h4
16.92x9.015,1,1,1,2,1,2,1,1,2,1,...,0,0,0,0,0,0,0,0,0,0
16.945x11.075,0,0,3,2,2,7,0,2,3,2,...,0,0,0,0,0,0,0,0,0,0
16.97x10.118,0,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16.939x12.132,1,0,1,0,4,6,1,4,3,1,...,0,0,0,0,0,0,0,0,0,0
16.949x13.055,0,0,0,3,0,2,0,2,4,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
spatial_locations = pd.DataFrame(index=Y.index)
spatial_locations['x'] = Y.index.str.split('x').str.get(0).map(float)
spatial_locations['y'] = Y.index.str.split('x').str.get(1).map(float)


In [5]:
spatial_locations['total_counts'] = Y.sum(1)
Y = Y.loc[spatial_locations.index]
X = spatial_locations[['x', 'y']]
scale = pd.read_csv('./GPcounts/data/MouseOB/scales_nb.txt', sep="\t") # load the file with the scale factors 


After filtering, the whole data-set contains 14,859 genes but for simplicity we run this example on the first 20 genes. This will take a few minutes. For the purposes of comparison, we provide a file (../data/MouseOB/SE_genes_comparison.csv) with the all spatially expressed genes found by running GPcounts on the whole data-set as well as the spatially expressed genes identified by SpatialDE package.



In [6]:
Y = Y.T
Y_run = Y.iloc[0:20,:] # select first 20 genes to run GPcounts
X = X[['x', 'y']]


###To run the scaled version of the GPcounts, where the negative binomial mean is multiplied by the pre-culculated scale factors, we should use the optional argument in the function Fit_GPcounts: 'scale=scale'. The 'scale' argument contains the file with the scale factors.



In [9]:
gene_name = []
gene_name = Y_run.index
likelihood = 'Negative_binomial' 
gp_counts = Fit_GPcounts(X,Y_run.loc[gene_name], scale = scale,safe_mode=False)


###We run the One_sample_test with negative binomial likelihood for the 20 genes. The results will show the following:

####Name of the gene
####Dynamic_model_log_likelihood - Likelihood of the dynamic model
####Constant_model_log_likelihood - Likelihood of the constant ('null') model
####log_likelihood_ratio - Ratio between the dynamic and constant models
####p value - statistical significance based on P value
####q value - Statistical significance based on the "adjusted p value" for the false discovery rate or q value


In [10]:
log_likelihood_ratio = gp_counts.One_sample_test(likelihood)
log_likelihood_ratio


  0%|          | 0/20 [00:00<?, ?it/s]2023-03-09 01:46:41.931789: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


  0%|          | 0/20 [00:01<?, ?it/s]


TypeError: in user code:

    File "/home/rs619065/miniconda3/envs/r-4.1/lib/python3.8/site-packages/gpflow/optimizers/scipy.py", line 120, in _tf_eval  *
        loss, grads = _compute_loss_and_gradients(
    File "/home/rs619065/miniconda3/envs/r-4.1/lib/python3.8/site-packages/gpflow/optimizers/scipy.py", line 223, in _compute_loss_and_gradients  *
        loss = loss_closure()
    File "/home/rs619065/miniconda3/envs/r-4.1/lib/python3.8/site-packages/check_shapes/integration/tf.py", line 57, in wrapped_method  *
        return wrapped_function(self, *args, **kwargs)
    File "/home/rs619065/miniconda3/envs/r-4.1/lib/python3.8/site-packages/check_shapes/decorator.py", line 120, in wrapped_function  *
        return func(*args, **kwargs)
    File "/home/rs619065/miniconda3/envs/r-4.1/lib/python3.8/site-packages/gpflow/models/training_mixins.py", line 64, in training_loss  *
        return self._training_loss()  # type: ignore[attr-defined]
    File "/home/rs619065/miniconda3/envs/r-4

In [ ]:
results = gp_counts.calculate_FDR(log_likelihood_ratio)
results


####Plot some spatially expressed genes from the data-set based on the q values provided in file 'SE_genes_comparison.csv'



In [ ]:
from matplotlib import rcParams
plt.rcParams["figure.figsize"] = [10,3]
m = ['Ptn(8.14e-10)', 'Cck(2.73e-12)', 'Doc2g(1.34e-08)']
for i, g in enumerate(['Ptn', 'Cck', 'Doc2g']):
    plt.subplot(1, 3, i + 1)
    plt.scatter(spatial_locations['x'], spatial_locations['y'], c=Y.T[g],cmap = 'summer')
    # plt.figure(figsize=(10,3))
    plt.title(m[i])
    plt.axis('equal')
plt.colorbar(ticks=[0,1],label = 'Gene Epxression')

plt.show()


####We plot two genes with very low expression which have been identified as spatially expressed by SpatialDE but not by GPcounts



In [ ]:
plt.rcParams["figure.figsize"] = [10,3]
n = ['Olfr635','Gna14', 'Taf7l']
for i, g in enumerate(['Olfr635','Gna14', 'Taf7l']):
    plt.subplot(1, 3, i + 1)
    plt.scatter(spatial_locations['x'], spatial_locations['y'], c=Y.T[g],cmap = 'summer')
    # plt.figure(figsize=(10,3))
    plt.title(n[i])
    plt.axis('equal')
plt.colorbar(label = 'Gene Epxression')
